In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.efficientnet import preprocess_input
from ultralytics import YOLO
import matplotlib.pyplot as plt

# Paths
CLASSIFICATION_MODEL_PATH = "classification.keras"
DETECTION_MODEL_PATH = "bboxes.pt"
IMAGE_FOLDER = "./testimages"  # Change to your image folder

# Load Models
clf_model = load_model(CLASSIFICATION_MODEL_PATH)
det_model = YOLO(DETECTION_MODEL_PATH)

# Parameters
IMG_SIZE = 512
THRESHOLD = 0.5

# Helper: preprocess single image for classification
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    old_size = img.shape[:2]
    ratio = float(IMG_SIZE) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])
    img_resized = cv2.resize(img, (new_size[1], new_size[0]))
    delta_w = IMG_SIZE - new_size[1]
    delta_h = IMG_SIZE - new_size[0]
    top, bottom = delta_h // 2, delta_h - delta_h // 2
    left, right = delta_w // 2, delta_w - delta_w // 2
    padded_img = cv2.copyMakeBorder(img_resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)
    padded_img = np.repeat(np.expand_dims(padded_img, -1), 3, axis=-1)
    padded_img = preprocess_input(padded_img.astype('float32'))
    return np.expand_dims(padded_img, axis=0)

# Inference loop
for img_name in os.listdir(IMAGE_FOLDER):
    img_path = os.path.join(IMAGE_FOLDER, img_name)
    processed_img = preprocess_image(img_path)

    if processed_img is None:
        print(f"Skipped unreadable image: {img_name}")
        continue

    # Classify
    prediction = clf_model.predict(processed_img)[0][0]
    print(f"\n{img_name} - Tumor Probability: {prediction:.4f}")

    if prediction < THRESHOLD:
        print("No tumor detected.")
    else:
        print("Tumor detected! Running YOLOv8 detection...")
        det_model.predict(source=img_path, save=True, conf=0.25)
        print(f"Results saved in /runs/detect/predict")
